# 1. Install Package

In [1]:
!pip install mediapipe opencv-python

# 2. Import Package

In [1]:
import cv2 # openCV 라이브러리 
import mediapipe as mp
import numpy as np

# 3. Calculate Angle

In [3]:
# def calculate_angle(a,b,c):
#     a = np.array(a) # First
#     b = np.array(b) # Mid
#     c = np.array(c) # End
    
#     radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0]) # 두 벡터 사이의 각도 구하기
#     angle = np.abs(radians*180.0/np.pi) # 라디안을 각도로 변환
    
#     if angle >180.0: # 각도가 180보다 크면 보정 / 팔은 360도 회전 x
#         angle = 360-angle
        
#     return angle 

In [2]:
def calculate_angle_3d(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    # Calculate the vectors from b to a and b to c
    vector1 = a - b
    vector2 = c - b

    # Calculate the dot product and the magnitudes of the vectors
    dot_product = np.dot(vector1, vector2)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)

    # Calculate the cosine of the angle between the vectors
    cosine_theta = dot_product / (magnitude1 * magnitude2)

    # Calculate the angle in radians
    radians = np.arccos(np.clip(cosine_theta, -1.0, 1.0))

    # Convert radians to degrees
    angle = np.degrees(radians)

    return angle

In [13]:
import numpy as np
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture("mp4/lunge.mp4")

count = 0
stage = None
left_stage = None
right_stage = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)
    
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]
            
            left_angle = calculate_angle_3d(left_hip, left_knee, left_ankle)
            right_angle = calculate_angle_3d(right_hip, right_knee, right_ankle)
            
            # 이미지에 계산된 각도 시각화
            cv2.putText(image, f"Left Angle: {left_angle:.2f}", 
                (int(left_shoulder[0] * image.shape[1]), int(left_shoulder[1] * image.shape[0])),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

            # cv2.putText(image, f"Right Angle: {right_angle:.2f}", 
            #     (int(right_shoulder[0] * image.shape[1]), int(right_shoulder[1] * image.shape[0])),
            #     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            
            if left_angle > 140:
                left_stage = "up"
            if left_angle < 110 and left_stage =='up':
                left_stage="down"
                count += 1

            # if right_angle > 160:
            #     right_stage = "up"
            # if right_angle < 100 and right_stage =='up':
            #     right_stage="down"
            #     count += 1
                       
        except:
            pass
        
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)      
        
        cv2.putText(image, str(count), 
                    (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)  
        
        cv2.putText(image, left_stage, 
                    (60, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA) 
        
        cv2.putText(image, right_stage, 
                    (60, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)  
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


KeyboardInterrupt: 